In [11]:
import onnxruntime as ort
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
net = ort.InferenceSession("../models/ncf/model-q.onnx")

In [13]:
movies = pd.read_csv("../models/ncf/movies.csv")
ratings = pd.read_parquet("../models/ncf/ratings.parquet", engine="pyarrow")
embeddings = np.load('../models/ncf/embeddings.npy')

### DB

In [14]:
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text, inspect
from tqdm import tqdm

# db_host = "movie-ratings.c7aw8qoquhqp.us-east-1.rds.amazonaws.com"
db_host = "movie-ratings.clweeg2y8lzr.ap-southeast-1.rds.amazonaws.com"
db_name = "postgres"
db_user = "postgres"
db_password = "postgres-root"
db_port = "5432"

In [15]:
try:
    # conn = psycopg2.connect(
    #     dbname=db_name,
    #     user=db_user,
    #     password=db_password,
    #     host=db_host,
    #     port=db_port
    # )
    # conn.close()
    engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    print("Successful")
except Exception as e:
    print("Failed:", e)

Successful


In [ ]:
def load_engine():
    db_host = "movie-ratings.clweeg2y8lzr.ap-southeast-1.rds.amazonaws.com"
    db_name = "postgres"
    db_user = "postgres"
    db_password = "postgres-root"
    db_port = "5432"

    engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    
    return engine

In [16]:
# for i in tqdm(range(0, len(ratings), 10000), desc="Uploading..."):
#     ratings.iloc[i:i+10000].to_sql('ratings', con=engine, if_exists='append', index=False)

In [17]:
# try:
#     # engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    
#     with engine.connect() as conn:
#         try:
#             conn.execute(text('CREATE INDEX IF NOT EXISTS "idx_movieId" ON ratings("movieId");'))
#             conn.execute(text('CREATE INDEX IF NOT EXISTS "idx_rating" ON ratings("rating");'))
#             conn.execute(text('CREATE INDEX IF NOT EXISTS "idx_movieId_rating" ON ratings("movieId", "rating");'))
#             conn.execute(text('CREATE INDEX IF NOT EXISTS "idx_rating_movieId" ON ratings("rating", "movieId");'))
#             conn.commit()
#             print("Successful")
#         except Exception as e:
#             print("Error:", e)
            
# except Exception as e:
#     print("Failed:", e)

In [18]:
try:
    # engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    
    inspector = inspect(engine)
    table_names = inspector.get_table_names()
    print("Tables:", table_names)
    
    for table_name in table_names:
        columns = inspector.get_columns(table_name)
        column_names = [col["name"] for col in columns]
        print("Columns:", column_names)
        
except Exception as e:
    print("Failed:", e)

Tables: ['ratings']
Columns: ['userId', 'movieId', 'rating']


In [19]:
try:
    # engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    
    inspector = inspect(engine)
    indexes = inspector.get_indexes("ratings")
    print("Indexes:", [index["name"] for index in indexes])
        
except Exception as e:
    print("Failed:", e)

Indexes: ['idx_movieId', 'idx_movieId_rating', 'idx_rating', 'idx_rating_movieId']


In [20]:
try:
    # engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    
    with engine.connect() as conn:
        
        # conn.execute(text("SET enable_indexscan = ON;"))
        
        query = "EXPLAIN ANALYZE SELECT * FROM ratings WHERE rating > 4.0;"
        df_explain = pd.read_sql(query, con=conn)
        
        query = "SELECT * FROM ratings WHERE rating > 4.0;"
        df = pd.read_sql(query, con=conn)
        
    print(df_explain)
    print(df.head())
        
except Exception as e:
    print("Failed:", e)

                                          QUERY PLAN
0  Seq Scan on ratings  (cost=0.00..603825.65 row...
1           Filter: (rating > '4'::double precision)
2                   Rows Removed by Filter: 24429627
3                            Planning Time: 0.216 ms
4                        Execution Time: 2339.222 ms
   userId  movieId  rating
0       1       30     5.0
1       1       32     5.0
2       1       80     5.0
3       1      111     5.0
4       1      166     5.0


### Main Function

In [ ]:
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

# ratings['user'] = user_encoder.fit_transform(ratings['userId'])
# ratings['movie'] = movie_encoder.fit_transform(ratings['movieId'])
# ratings.drop(columns=['user', 'movie'], inplace=True)

user_encoder.fit_transform(ratings['userId'])
movie_encoder.fit_transform(ratings['movieId'])

In [5]:
all_genres = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [6]:
def recommend_movies(user_id, net, movies, user_encoder, movie_encoder, ratings, top_n=10):
    try:
        user_encoded = user_encoder.transform([user_id])[0]
    except:
        print("User ID Invalid")
        return pd.DataFrame()
    
    all_movies = np.arange(84432)

    user_data = ratings[ratings['userId'] == user_id]
    rated_movie_ids = user_data['movieId'].unique()
    rated_movies = movie_encoder.transform(rated_movie_ids)
    
    movies_to_predict = np.setdiff1d(all_movies, rated_movies).astype(np.int64)
    
    predicted_ratings = []
    movie_ids = []
    
    batch_size = 1024
    for i in range(0, len(movies_to_predict), batch_size):
        batch_movies = movies_to_predict[i:i+batch_size]
        batch_users = np.array([user_encoded] * len(batch_movies), dtype=np.int64)
        inputs = {
            'user_input': batch_users,
            'movie_input': batch_movies
        }
        outputs = net.run(['rating_output'], inputs)[0]
        predicted_ratings.extend(outputs.flatten())
        batch_movie_ids = movie_encoder.inverse_transform(batch_movies)
        movie_ids.extend(batch_movie_ids)
    
    predictions_df = pd.DataFrame({
        'movieId': movie_ids,
        'predicted_rating': predicted_ratings
    })
    
    recommendations = predictions_df.merge(movies[['movieId', 'title']], on='movieId')
    recommendations = recommendations.sort_values(by='predicted_rating', ascending=False).head(top_n)
    recommendations["predicted_rating"] = np.floor(recommendations["predicted_rating"] * 100) / 100
    
    recommendations = recommendations.reset_index(drop=True)
    recommendations.index += 1
    
    recommendations.rename(columns={
    "movieId": "Movie ID",
    "title": "Title",
    "predicted_rating": "Predicted Rating"
    }, inplace=True)
    recommendations["Year"] = recommendations["Title"].str.extract(r"\((\d{4})\)")
    recommendations["Title"] = recommendations["Title"].str.replace(r" \(\d{4}\)", "", regex=True)
    recommendations = recommendations[["Movie ID", "Title", "Year", "Predicted Rating"]]
    return recommendations

In [ ]:
user_id=2
print(f"Recommended Movies for: User {user_id}")
user_recommendations = recommend_movies(
    user_id=user_id,
    net=net,
    movies=movies,
    user_encoder=user_encoder,
    movie_encoder=movie_encoder,
    ratings=ratings,
    top_n=10
)
display(user_recommendations)

In [8]:
def recommend_similar_movies_id(movie_id, movies, movie_encoder, embeddings, top_n=10):
    
    if movie_id not in movies['movieId'].values:
        print("Movie ID Invalid")
        return pd.DataFrame()
    # print(f"Recommended Movies Similar to: {movies[movies['movieId'] == movie_id]['title'].values[0]}")
    
    movie_encoded = movie_encoder.transform([movie_id])[0]
    target_embedding = embeddings[movie_encoded].reshape(1, -1)
    
    similarities = cosine_similarity(target_embedding, embeddings).flatten()
    similar_indices = similarities.argsort()[-(top_n + 1):-1][::-1]
    similar_movie_ids = movie_encoder.inverse_transform(similar_indices)
    
    recommendations = movies[movies['movieId'].isin(similar_movie_ids)][['movieId', 'title']]
    recommendations['similarity'] = similarities[similar_indices]
    recommendations["similarity"] = np.ceil(recommendations["similarity"] * 1000) / 100
    
    recommendations = recommendations.reset_index(drop=True)
    recommendations.index += 1
    
    recommendations.rename(columns={
    "movieId": "Movie ID",
    "title": "Title",
    "similarity": "Cosine Similarity"
    }, inplace=True)
    recommendations["Year"] = recommendations["Title"].str.extract(r"\((\d{4})\)")
    recommendations["Title"] = recommendations["Title"].str.replace(r" \(\d{4}\)", "", regex=True)
    recommendations = recommendations[["Movie ID", "Title", "Year", "Cosine Similarity"]]
    return recommendations

In [ ]:
movie_id=1
similar_movies = recommend_similar_movies_id(
    movie_id=movie_id,
    movies=movies,
    movie_encoder=movie_encoder,
    embeddings=embeddings,
    top_n=10
)
display(similar_movies)

In [10]:
def recommend_similar_movies_name(movie_name, movies, movie_encoder, embeddings, top_n=10):
    
    movie_name = movie_name.strip()
    matching_movies = movies[movies['title'].str.contains(movie_name, case=False, regex=True)]
    if matching_movies.empty:
        print("Movie Name Invalid")
        return pd.DataFrame()
    movie_id = matching_movies.iloc[0]['movieId']
    # print(f"Recommended Movies Similar to: {movies[movies['movieId'] == movie_id]['title'].values[0]}")
    
    movie_encoded = movie_encoder.transform([movie_id])[0]
    target_embedding = embeddings[movie_encoded].reshape(1, -1)
    
    similarities = cosine_similarity(target_embedding, embeddings).flatten()
    similar_indices = similarities.argsort()[-(top_n + 1):-1][::-1]
    similar_movie_ids = movie_encoder.inverse_transform(similar_indices)
    
    recommendations = movies[movies['movieId'].isin(similar_movie_ids)][['movieId', 'title']]
    recommendations['similarity'] = similarities[similar_indices]
    recommendations["similarity"] = np.ceil(recommendations["similarity"] * 1000) / 100
    
    recommendations = recommendations.reset_index(drop=True)
    recommendations.index += 1
    
    recommendations.rename(columns={
    "movieId": "Movie ID",
    "title": "Title",
    "similarity": "Cosine Similarity"
    }, inplace=True)
    recommendations["Year"] = recommendations["Title"].str.extract(r"\((\d{4})\)")
    recommendations["Title"] = recommendations["Title"].str.replace(r" \(\d{4}\)", "", regex=True)
    recommendations = recommendations[["Movie ID", "Title", "Year", "Cosine Similarity"]]
    return recommendations

In [ ]:
movie_name="Godfather"
similar_movies = recommend_similar_movies_name(
    movie_name=movie_name,
    movies=movies,
    movie_encoder=movie_encoder,
    embeddings=embeddings,
    top_n=10
)
display(similar_movies)

In [12]:
def recommend_by_genre_pop(genres, movies, ratings, top_n=10):
    valid_genres = sorted(all_genres)
    for genre in genres:
        if genre not in valid_genres:
            print(f"Genre '{genre}' Invalid. Valid Genres: {valid_genres}")
            return pd.DataFrame()
    
    filtered_movies = movies
    for genre in genres:
        filtered_movies = filtered_movies[filtered_movies[genre] == 1]
    
    if filtered_movies.empty:
        print("No Movies Found w/ Specified Genre(s)")
        return pd.DataFrame()
    
    popularity = ratings.groupby('movieId').size().reset_index(name='rating_count')
    
    recommendations = filtered_movies.merge(popularity, on='movieId', how='left').fillna({'rating_count': 0})
    recommendations = recommendations.sort_values(by='rating_count', ascending=False)
    recommendations = recommendations[['movieId', 'title', 'rating_count']].head(top_n)
    recommendations["rating_count"] = recommendations["rating_count"].astype(int)
    
    recommendations = recommendations.reset_index(drop=True)
    recommendations.index += 1
    
    recommendations.rename(columns={
    "movieId": "Movie ID",
    "title": "Title",
    "rating_count": "Total Rating"
    }, inplace=True)
    recommendations["Year"] = recommendations["Title"].str.extract(r"\((\d{4})\)")
    recommendations["Title"] = recommendations["Title"].str.replace(r" \(\d{4}\)", "", regex=True)
    recommendations = recommendations[["Movie ID", "Title", "Year", "Total Rating"]]
    
    return recommendations

In [ ]:
selected_genres = ['Action', 'Adventure']
print(f"Recommended Movies w/ Genres: {', '.join(selected_genres)}")
genre_recommendations = recommend_by_genre_pop(
    genres=selected_genres,
    movies=movies,
    ratings=ratings,
    top_n=10
)
display(genre_recommendations)

In [14]:
def recommend_by_genre_avg(genres, movies, ratings, top_n=10):
    valid_genres = sorted(all_genres)
    for genre in genres:
        if genre not in valid_genres:
            print(f"Genre '{genre}' is not recognized. Valid genres are: {valid_genres}")
            return pd.DataFrame()
    
    filtered_movies = movies
    for genre in genres:
        filtered_movies = filtered_movies[filtered_movies[genre] == 1]
    
    if filtered_movies.empty:
        print("No movies found with the specified genres.")
        return pd.DataFrame()
    
    rating_stats = ratings.groupby('movieId').agg(
        average_rating=('rating', 'mean'),
        rating_count=('rating', 'size')
    ).reset_index()
    
    recommendations = filtered_movies.merge(rating_stats, on='movieId', how='left').fillna({'average_rating': 0, 'rating_count': 0})
    recommendations = recommendations[recommendations['rating_count'] >= 10]
    recommendations = recommendations.sort_values(by='average_rating', ascending=False)
    recommendations = recommendations[['movieId', 'title', 'average_rating', 'rating_count']].head(top_n)
    recommendations["average_rating"] = np.ceil(recommendations["average_rating"] * 1000) / 1000
    recommendations["rating_count"] = recommendations["rating_count"].astype(int)
    
    recommendations = recommendations.reset_index(drop=True)
    recommendations.index += 1
    
    recommendations.rename(columns={
    "movieId": "Movie ID",
    "title": "Title",
    "average_rating": "Average Rating",
    "rating_count": "Total Rating"
    }, inplace=True)
    recommendations["Year"] = recommendations["Title"].str.extract(r"\((\d{4})\)")
    recommendations["Title"] = recommendations["Title"].str.replace(r" \(\d{4}\)", "", regex=True)
    recommendations = recommendations[["Movie ID", "Title", "Year", "Average Rating", "Total Rating"]]
    
    return recommendations

In [ ]:
selected_genres = ['Horror', 'Adventure']
print(f"Recommended Movies w/ Genres: {', '.join(selected_genres)}")
genre_recommendations = recommend_by_genre_avg(
    genres=selected_genres,
    movies=movies,
    ratings=ratings,
    top_n=10
)
display(genre_recommendations)

In [16]:
# def recommend_combined(movie_id, genres, movies, movie_encoder, embeddings, ratings, top_n=10):
#     if movie_id not in movies['movieId'].values:
#         print("Movie ID Invalid")
#         return pd.DataFrame()
    
#     movie_encoded = movie_encoder.transform([movie_id])[0]
#     target_embedding = embeddings[movie_encoded].reshape(1, -1)
    
#     similarities = cosine_similarity(target_embedding, embeddings).flatten()
    
#     similar_indices = similarities.argsort()[-(top_n * 2 + 1):-1][::-1]
#     similar_movie_ids = movie_encoder.inverse_transform(similar_indices)
#     similar_movies = movies[movies['movieId'].isin(similar_movie_ids)].copy()
#     similar_movies['similarity'] = similarities[similar_indices]
    
#     for genre in genres:
#         if genre in all_genres:
#             similar_movies = similar_movies[similar_movies[genre] == 1]
#         else:
#             print(f"Genre '{genre}' Invalid.")
#             return pd.DataFrame()
    
#     if similar_movies.empty:
#         print("No Movies Found w/ Specified Genre(s)")
#         return pd.DataFrame()
    
#     popularity = ratings.groupby('movieId').size().reset_index(name='rating_count')
#     similar_movies = similar_movies.merge(popularity, on='movieId', how='left').fillna({'rating_count': 0})
    
#     similar_movies = similar_movies.sort_values(by=['similarity', 'rating_count'], ascending=[False, False])
#     recommendations = similar_movies[['movieId', 'title', 'similarity', 'rating_count']].head(top_n)
    
#     return recommendations.reset_index(drop=True)

In [17]:
# movie_id = 100
# selected_genres = ['Action']
# print(f"Recommended Movies Similar to: {movies[movies['movieId'] == movie_id]['title'].values[0]} w/ Genres: {', '.join(selected_genres)}")
# combined_recommendations = recommend_combined(
#     movie_id=movie_id,
#     genres=selected_genres,
#     movies=movies,
#     movie_encoder=movie_encoder,
#     embeddings=embeddings,
#     ratings=ratings,
#     top_n=10
# )
# display(combined_recommendations)

In [18]:
def recommend_combined_mix(movie_name, genres, movies, movie_encoder, embeddings, ratings, top_n=10):
    similar_movies = recommend_similar_movies_name(movie_name, movies, movie_encoder, embeddings, top_n=top_n*2)
    if similar_movies.empty:
        return pd.DataFrame()
    
    similar_movie_ids = similar_movies["Movie ID"].astype(int).values
    
    filtered_movies = movies[movies['movieId'].isin(similar_movie_ids)].copy()
    for genre in genres:
        if genre in all_genres:
            filtered_movies = filtered_movies[filtered_movies[genre] == 1]
        else:
            print(f"Genre '{genre}' Invalid.")
            return pd.DataFrame() 
    
    if filtered_movies.empty:
        print("No Movies Found w/ Specified Genre(s)")
        return pd.DataFrame()
    
    popularity = ratings.groupby('movieId').size().reset_index(name='rating_count')
    filtered_movies = filtered_movies.merge(popularity, on='movieId', how='left').fillna({'rating_count': 0})
    
    filtered_movies = filtered_movies.merge(similar_movies[['Movie ID', 'Cosine Similarity']],
                                            left_on='movieId', right_on='Movie ID', how='left')
    
    filtered_movies = filtered_movies.sort_values(by=['Cosine Similarity', 'rating_count'], ascending=[False, False])
    
    recommendations = filtered_movies[['movieId', 'title', 'Cosine Similarity', 'rating_count']].head(top_n)
    
    recommendations.rename(columns={
        'movieId': 'Movie ID',
        'title': 'Title',
        'rating_count': 'Total Rating'
    }, inplace=True)
    recommendations["Year"] = recommendations["Title"].str.extract(r"\((\d{4})\)")
    recommendations["Title"] = recommendations["Title"].str.replace(r" \(\d{4}\)", "", regex=True)
    recommendations = recommendations[["Movie ID", "Title", "Year", "Cosine Similarity", "Total Rating"]]
    recommendations = recommendations.reset_index(drop=True)
    recommendations.index += 1
    return recommendations

In [ ]:
movie_name = "Godfather"
selected_genres = ['Action']
combined_recommendations = recommend_combined_mix(
    movie_name=movie_name,
    genres=selected_genres,
    movies=movies,
    movie_encoder=movie_encoder,
    embeddings=embeddings,
    ratings=ratings,
    top_n=10
)
display(combined_recommendations)